In [107]:
# alpha:finish consulting; beta:new coming;
# overall timer format: (timepoint, event, stationid, serverid)

class Station:
    def __init__(self, stationid, overall_timer, process_times, n_server=0, n_buffer=0):
        self.stationid = stationid
        self.overall_timer = overall_timer
        self.s = n_server
        self.c = n_buffer
        self.wait = False
        self.status = [0, [0]*n_server] # 1 for occupied
        self.process_times = process_times
        self.next_station = None
    
    def setnext(self, next_station):
        self.next_station = next_station
        
    def printout(self):
        print("time:{:.04f}".format(self.overall_timer[0][0]), " \tstation "+str(self.stationid)+":", self.status)
    
    def update(self, timeslot):
        event= timeslot[1]
        if event == 'alpha':
            serverid = timeslot[3]
            self.alpha(serverid, timeslot)
        else:
            self.beta()
        del(self.overall_timer[0])
        self.overall_timer.sort(key= lambda k:k[0])
        
    def alpha(self, serverid, timeslot):
        if self.next_station == None:
            if self.status[0] > 0: # wqiting line is not empty
                self.status[0] -= 1
                try:
                    self.overall_timer.append((self.overall_timer[0][0]+self.process_times[0], 'alpha', self.stationid, serverid))
                    del(self.process_times[0])
                    self.printout()
                except:
                    print("empty process time list for station", self.stationid, "server", serverid)
            else:
                self.status[1][serverid] = 0
                self.printout()
                return True
        else:
            if self.next_station.beta():
                if self.status[0] > 0: # wqiting line is not empty
                    self.status[0] -= 1
                    try:
                        self.overall_timer.append((self.overall_timer[0][0]+self.process_times[0], 'alpha', self.stationid, serverid))
                        del(self.process_times[0])
                        self.printout()
                    except:
                        print("empty process time list for station", self.stationid, "server", serverid)
                else:
                    self.status[1][serverid] = 0
                    self.printout()
                    return True
            else:
                del(self.overall_timer[0])
                for i in range(len(self.overall_timer)):
                    if self.overall_timer[i][2]==self.stationid+1:
                        self.overall_timer.append((self.overall_timer[i][0], 'alpha', self.stationid, serverid))
                        return True
                self.overall_timer.append((self.overall_timer[0][0], 'alpha', self.stationid, serverid))
            
    def beta(self):
        if self.status[0] > 0: 
            if self.status[0]<self.c:
                self.status[0]+=1
                self.printout()
                return True
            else:
                return False
        else:
            i = 0
            while i < self.s:
                if self.status[1][i]==0:
                    self.status[1][i]=1
                    self.overall_timer.append((self.overall_timer[0][0]+self.process_times[0], 'alpha', self.stationid, i))
                    self.process_times.remove(self.process_times[1])
                    break
                else:
                    i += 1
            if i == self.s:
                self.status[0]+=1
            self.printout()
            return True
        

In [108]:
import numpy as np
class simulation:
    def run():
        timer = []
        t = np.random.poisson(2.1, 20)
        for i in range(20):
            timer.append((t[i], 'beta', 0))
        timer.sort(key= lambda k:k[0])
            
        stations = []
        stations.append(Station(stationid=0, overall_timer=timer, process_times=list(np.random.exponential(2.9, 20)), n_server=1, n_buffer=5))
        stations.append(Station(stationid=1, overall_timer=timer, process_times=list(np.random.exponential(2.2, 20)), n_server=1, n_buffer=6))
        stations.append(Station(stationid=2, overall_timer=timer, process_times=list(np.random.exponential(2.2, 20)), n_server=1, n_buffer=6))
        stations.append(Station(stationid=3, overall_timer=timer, process_times=list(np.random.exponential(2.8, 20)), n_server=1, n_buffer=3))
        stations[2].setnext(stations[3])
        stations[1].setnext(stations[2])
        stations[0].setnext(stations[1])
        
        tt = 0
        while len(timer)>0:
            # print(timer)
            tt = timer[0][0]
            stations[timer[0][2]].update(timer[0])
        return tt
            

In [109]:
simulation.run()

time:0.0000  	station 0: [0, [1]]
time:0.0000  	station 0: [1, [1]]
time:1.0000  	station 0: [2, [1]]
time:1.0000  	station 0: [3, [1]]
time:1.0000  	station 0: [4, [1]]
time:1.0000  	station 0: [5, [1]]
time:1.8888  	station 1: [0, [1]]
time:1.8888  	station 0: [4, [1]]
time:2.0000  	station 0: [5, [1]]
time:3.6855  	station 2: [0, [1]]
time:3.6855  	station 1: [0, [0]]
time:3.7777  	station 1: [0, [1]]
time:3.7777  	station 0: [4, [1]]
time:3.8129  	station 3: [0, [1]]
time:3.8129  	station 2: [0, [0]]
time:4.0000  	station 0: [5, [1]]
time:4.3816  	station 1: [1, [1]]
time:4.3816  	station 0: [4, [1]]
time:5.0000  	station 0: [5, [1]]
time:5.5744  	station 2: [0, [1]]
time:5.5744  	station 1: [0, [1]]
time:5.7018  	station 3: [1, [1]]
time:5.7018  	station 2: [0, [0]]
time:5.7239  	station 3: [0, [1]]
time:5.8611  	station 1: [1, [1]]
time:5.8611  	station 0: [4, [1]]
time:7.3710  	station 2: [0, [1]]
time:7.3710  	station 1: [0, [1]]
time:7.4984  	station 3: [1, [1]]
time:7.4984  	

38.89488937915299